# Inteligência Artificial e Redes Neurais 

## Atividade


In [0]:
# Carregar datasets
!wget http://www.data2learning.com/ml_datasets/dataset1_1.pkl
!wget http://www.data2learning.com/ml_datasets/dataset1_2.pkl
!wget http://www.data2learning.com/ml_datasets/dataset2.pkl
!wget http://www.data2learning.com/ml_datasets/dataset3.pkl
!wget http://www.data2learning.com/ml_datasets/sentimentdataset.pkl

In [0]:
!ls

# Tutorial

A seguir está descrito todo o código necessário para executar os problemas práticos da atividade. 

## Instanciando os Modelos


### KNN
```python
knnmodel = KNeighborsClassifier(n_neighbors=3)
```

O parâmetro **n_neightbors** corresponde ao valor de K vizinhos do algoritmo KNN. Ele pode ser variado.

### Regressão Linear
```python
lm = LinearRegression()
```

### Árvore de Decisão
```python
tree_model = tree.DecisionTreeClassifier(criterion='entropy',random_state=0)
```

### SVM
```python
clfsvm = svm.SVC(kernel=kernel, C=C, random_state=0)
```

Pode variar o kernel ('linear', 'poly', 'rbf', 'sigmoid') e o valor de C. No kernel *rbf* pode variar o parâmetro *gamma* e no *poly* pode variar o parâmetro *degree*. 

### Naive Bayes 


naivemodel = GaussianNB()


### Rede Neural Perceptron
```python
perceptron_ = perceptron.Perceptron(n_iter=100, eta0=0.001, random_state=0, verbose=False)
```

Pode variar o número de iterações (*n_iter*) e a taxa de aprendizagem (*eta0*).

### Rede Neural MLP 

```python
mlp_ = MLPClassifier(
    hidden_layer_sizes=(256), 
    activation='relu', 
    batch_size=10, 
    verbose=False, 
    max_iter=120, 
    learning_rate_init=0.001,
    random_state=0,
    alpha=0.1
)
```
Pode variar o tamanho e número de camadas (*hidden_layer_size*):

* (256): 1 camada com 256 neurônios;
* (256, 256): 2 camadas com 256 neurônios cada;
* ... 

Pode variar a função de ativação (*activation*):

* Valores possíveis: 'identity', 'logistic', 'tanh', 'relu'

Pode variar o número de iterações (*max_iter*). 

Pode variar a taxa de aprendizagem (*learning_rate_init*)


## Treinando os Modelos

Um modelo pode ser treinado utilizando o método FIT. Isso serve para qualquer modelo do Scikit-Learn. 

```python
modelo.fit(X, Y) #X = atributos, Y = a classe (ou labels)
```

## Predição

```python
modelo.predict(Z)
```

Z é uma instância de teste. Por exemplo, se sua base possui três atributos que correspondem as características daquelas instâncias, o método predict tem que receber três atributos também na hora de fazer a predição. Ou seja, Z seria algo do tipo [10, 2.4, 3.1], por exemplo.

## Divisão Treino/Teste

```python
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=4)
```

onde *test_size* é a porcentagem de divisão. No exemplo, 20% da base é de teste e o restante é de treinamento. *X* são os dados (instâncias e atributos) e *y* a classe de cada instância. *random_state* é o controle do aleatório. Deve-se manter 4. 


## Avaliação

O modelo pode ser avaliado usando o método *score*.

```python
modelo.score(X, y)
```

Para que este método funcione, o modelo deve ser treinado antes (usando o *fit*).


## Validação Cruzada

Uma outra forma de analisarmos o modelo construído é utilizando a validação cruzada. Para usá-la utilizamos: 

```python 
scores = cross_val_score(MODELO, X, Y, cv=10, scoring='accuracy')
```

* onde o MODELO é a instância do modelo que estamos criando (Arvore de Decisão, Regressão Linear, Rede Neural etc)
* X e Y são os dados com os atributos (X) e somente a coluna de resultado (Y).
* cv = quantidade de folds 
* scoring: métrica de avaliação. Utiilze **r2** se estiver trabalhando com Regressão Linear.

Esse método já faz o processo de treinamento e teste ao mesmo tempo. Não é necessário utilizar o **Fit** quando chamamos ele. 

Para obter a média do score basta utilizar: 

```python
scores.mean()
```

## Python

Para fazer um for de um até N utilize a seguinte notação: 

```python
for n in range(1, N+1):
    print n
```

Se desejar imprimir algo no console, utilize o método **print**

```python
print("Teste")
print(X)
```

In [0]:
# Instalação do NLKT

import nltk
nltk.download()

In [0]:
# Imports Necessários

#Bibliotecas
import nltk
import re
import pandas as pd
import numpy as np 
import unicodedata
import matplotlib.pyplot as plt
from sklearn.externals import joblib
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

# Modelos 
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn import tree # Arvore de Decisao
from sklearn import svm # SVM
from sklearn.naive_bayes import GaussianNB # Naive Bayes
from sklearn.ensemble import RandomForestClassifier # Random Forest
from sklearn.linear_model import perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans

#Avaliação
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

#stopwords
portuguese_stop = stopwords.words(['portuguese'])

%matplotlib inline

### Funções de Suporte 1

As funções a seguir são de suporte para diversas tarefas que foram realizadas na realização da prova. Para a realização da prova não é preciso usá-las. No entanto, a célula deve ser executada para garantir a execução da prova sem erros.

In [0]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

def tokenize_only(text):
    twtk = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = twtk.tokenize(text)
    #[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        token = re.sub(r"http\S+", "", token)
        token = re.sub(r"[...]","", token)
        token = strip_accents(token)
        if token not in portuguese_stop and len(token) >= 2:
            
            filtered_tokens.append(token)
            #if re.search('[a-zA-Z]', token):
            #    filtered_tokens.append(token)
    return filtered_tokens

## **Questão 01:** 

Foram estudados dois tipos de aprendizagem supervisionada: classificação e regressão. Sobre este assunto, responda o que se pede: 

**a)** Diferencie **classificação** e **regressão**. Deixe claro na explicação, o que de fato diferencia as duas técnicas principalemente em relação aos problemas em que elas são aplicadas. 

**b)** A seguir são apresentadas duas bases. Sua tarefa é analisar as informações da duas e aplicar o algoritmo correto para cada uma delas entre os algoritmos de **regressão linear** e **knn**. Na aplicação do algoritmo utilize a validação cruzada com 5 folds. Na aplicação da regressão linear reporte o valor da métrica **r2** e no caso do knn reporte a acurácia.

### Base 1

A base 1 é referenciada pelo trabalho "Predicting social media performance metrics and evaluation of the impact on brand building: A data mining approach"(Moro, 2016) e tem como objetivo prever a evolução de um post no facebook a partir de um conjunto de métricas coletadas dos posts. Na base, as *features* correspondem as métricas e o *label* (a coluna que deseja-se prever) corresponde a quantidade de iteração (comentários + curtidas +  compartilhamentos) de um post. 

In [0]:
dataset1_1 = joblib.load("dataset1_1.pkl")
dataset1_1['full'].head()

### Base 2

A base 2 é referenciada pelo trabalho "Phishing Detection based Associative Classification Data Mining" (Abdelhamid et al., 2014) e tem como objetivo identificar se um website sofreu um ataque de *pishing*. De forma geral, *Phishing* é um tipo de ataque que faz com que sites criados sejam substituídos por sites falsos, fazendo com que usuários passem informações sigilosas como CPF, logins, e-mails, senhas para criminosos. Nesta base, cada instância é um website que é classificado como Legitimate (1: site legítimo), Suspicious (0: suspeito) ou phishy (-1: sofreu pishing).

In [0]:
dataset1_2 = joblib.load("dataset1_2.pkl")
dataset1_2['full'].head()

In [0]:
# Imports das Bases

# Base 1

X_base1 = dataset1_1['X']
Y_base1 = dataset1_1['y']

# Base 2

X_base2 = dataset1_2['X']
Y_base2 = dataset1_2['y']

In [0]:
print(X_base1.shape, Y_base1.shape)
print(X_base2.shape, Y_base2.shape)

In [0]:
# Insira a resposta da questão 1 a partir daqui

## **Questão 02**: 

Uma árvore de decisão permite classificar um conjunto de dados a partir da divisão do espaço de busca de acordo com os valores dos atributos. Para cada nó da árvore, um atributo deve ser escolhido de forma que melhor separe o conjunto de dados. 

Sabendo que o **Ganho de Informação** é dado por: 

$GI(S,A) = E(S) - \sum_{v \in Valores(A)}{\frac{S_v}{S}E(S_v)}$, 

e a **Entropia** é dada por: 

$E = \sum_{i}^{c}{-p_i\log_2{p_i}}$

Você deve mostrar a partir do cálculo do Ganho de Informação e da Entropia qual o melhor atributo para ser utilizado como raiz da árvore.

**OBS: é obrigatório apresentar os cálculos realizados.**

In [0]:
dataset2 = joblib.load("dataset2.pkl")
dataset2

*Insira a resposta da questão 2 aqui* 

## **Questão 03:** 

Os algoritmos de SVM e Naive Bayes são dois clássicos algoritmos supervisionados. Ambos se valem de conceitos interessantes da matemática para construir os modelos de aprendizagem. Sobre estes algoritmos, responda:

**a)** Explique, brevemente, os conceitos por trás destes dois algoritmos. Deixe claro na sua explicação quais conceitos matemáticos estudados em sala de aula são utilizados na construção de cada um dos modelos.

**b)** Aplique os dois algoritmos na base a seguir. Analise os resultados obtidos nas configurações padrões do algoritmo e depois modifique os parâmetros do melhor algoritmo para se obter um resultado melhor do que o anterior. Os testes devem ser realizados utilizando validação cruzada de 5 folds. A análise deve ser feita a partir destes valores. 

In [0]:
dataset3 = joblib.load("dataset3.pkl")
X = dataset3['X']
Y = dataset3['Y']

In [0]:
X.head()

In [0]:
# Responda a questão 3 a partir daqui

## Questão 04

Você foi contratado por uma empresa para construir um sistema que permite analisar automaticamente um conjunto de tweets e determinar se o mesmo é classificado como positivo ou negativo. Sabendo que você tem uma base de dados pré-classificada de tweets positivos e tweets negativos em português. Responda o que se pede.

**a)** Proponha um conjunto de tarefas que você deve realizar para atender o objetivo para o qual você foi contratado. Deixe claro cada um dos procedimentos que serão tomados e porque usa-lo. Deixe claro como você pode maximizar as chances de de construir um sistema como boa acurácia dentre os modelos que foram disponibilizados no início desta prova.

*Insira sua resposta a partir daqui* 

------
O código a seguir carrega a base de dados disponibilizada para executar os itens **b** e **c**. Leia com atenção a descrição completa da base. O seu entendimento é essencial para a realização das tarefas que seguem. 

In [0]:
# Carrega a base de dados

'''
    A base de dados é composta por três componentes: 
    
     sentiment_dataset['X']: matrix de instâncias e termos utilizado para treinamento do modelo. 
     sentiment_dataset['y']: vetor de classes que indica a classe real de cada uma das instâncias 
                             da base de treinamento. Na base, 0 é negativo e 1 é positivo. 
     sentiment_dataset['vector']: modelo de vetorização da base de dados original. Isso deve ser utilizado 
                                 na hora de analisar os novos dados como descrito a seguir.

'''

sentiment_dataset = joblib.load("sentimentdataset.pkl")
X = sentiment_dataset['X']
y = sentiment_dataset['y']
vector = sentiment_dataset['vector']

**b)** Baseado no que você descreveu no **item a**, implemente um passo a passo para a construação de um modelo que permite reconhecer testes. Implemente alguns métodos, modifique os parâmetros e apresente como resposta a acurácia do melhor modelo. 

**c)** Utilize o melhor modelo gerado no **item b** e use-o para analisar os textos a seguir. Para cada texto deve-se indicar a classe que o modelo classifica cada um. Na classificação, 1 é positivo e 0 é negativo. 

In [0]:
'''
    Os textos a seguir devem ser classificados a partir do modelo que foi escolhido na questão anterior. Antes 
    de fazer a predição, deve-se gerar X_new_ que nada mais é do que sua base de teste. O código a seguir transforma
    tal texto em uma matriz onde as linhas são as instâncias e as colunas as palavras que foram utilizadas na base
    de treinamento. 
'''

new_text = [
    'eu realmente acho que os produtos da apple sao otimos :)', 
    'eu gosto de cantar <3', 
    'triste noticia',
    'odeio esse produto :(',
    'eu nao gosto da saga crepusculo'
]

X_new_ = vector.transform(new_text).toarray()